## Introduction

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return back has become automatic. Through these systems, user is able to easily rent a bike from a particular position and return back at another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousands bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

In this notebook, we'll be building a simple regression model to predict hourly bike rentals using a popular dataset. There are two goals for this notebook:

1. Show how XGBoost can be used to build a regression model
2. Describe various SHAP visualizations we can build to understand our model

## Background

This dataset contains daily counts of rented bicycles from the bicycle rental company Capital-Bikeshare in Washington D.C., along with weather and seasonal information. The goal is to predict how many bikes will be rented depending on the weather and the day. The data can be downloaded from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).

Features available in the dataset: 

- Count of bicycles including both casual and registered users. The count is used as the target in the regression task.
- The season, either spring, summer, fall or winter.
- Indicator whether the day was a holiday or not.
- The year, either 2011 or 2012.
- The hour of day : 0 to 23
- Number of days since the 01.01.2011 (the first day in the dataset).
- Indicator whether the day was a working day or weekend.
- The weather situation on that day. One of:
    - clear, few clouds, partly cloudy, cloudy
    - mist + clouds, mist + broken clouds, mist + few clouds, mist
    - light snow, light rain + thunderstorm + scattered clouds, light rain + scattered clouds
    - heavy rain + ice pallets + thunderstorm + mist, snow + mist
- Temperature in degrees Celsius.
- Relative humidity in percent (0 to 100).
- Wind speed in km per hour.

## Preparation and imports

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import pickle as pkl                              # For serializing/deserializing model
import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
import os                                         # For manipulating filepath names
from sklearn.model_selection import train_test_split   # For splitting the dataset

In [ ]:
# Define IAM role: automatically get the role if run on a SageMaker instance
import boto3
import re
from sagemaker import get_execution_role

# role = get_execution_role()
role = "arn:aws:iam::398557468931:role/service-role/AmazonSageMaker-ExecutionRole-20190626T094141"
region = boto3.Session().region_name

In [ ]:
#### Define S3 details for storing the dataset ######

BUCKET = 'xgboost-examples-1'
PREFIX = 'sagemaker/DEMO-bike-shap'

## Data

We'll use the dataset available from UCI's ML Repository and upload to S3 after some transformations.

In [ ]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
!unzip -o Bike-Sharing-Dataset.zip

In [ ]:
data = pd.read_csv('hour.csv')
print(data.shape)
data.head()

### Transformations

1. Replace column names with meaningful names
2. One-hot encode categorical columns

In [ ]:
data.rename(columns={'weathersit':'weather',
                     'mnth':'month',
                     'hr':'hour',
                     'hum': 'humidity',
                     'cnt':'count'},
            inplace=True)
# drop columns that don't add value or are directly correlated to output 
data = data.drop(['registered', 'casual', 'instant','dteday','yr'], axis=1)

# encode all categorical columns
categorical_columns = ['season', 'month', 'hour', 'holiday', 'weekday', 'weather']
categorical_col_values = {'season': ['spring', 'summer', 'fall', 'winter'], 
                          'month': ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'],
                          'hour': range(24), 
                          'holiday': ['no', 'yes'], 
                          'weekday': ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat'], 
                          'weather': ['clear', 'mist', 'snow', 'rain']}

data_dummy = pd.get_dummies(data, prefix=categorical_columns, columns=categorical_columns, drop_first=True)
data_dummy.head()

In [ ]:
y = data_dummy.loc[:, "count"]
X = data_dummy.drop("count", axis=1)
# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Use 'csv' format to store the data
# The first column is expected to be the output column
pd.concat([y_train, X_train], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([y_test, X_test], axis=1).to_csv('test.csv', index=False, header=False)

The input files to XGBoost cannot contain headers. Hence, we use a separate file to store the feature names ensuring our analysis contains meaningful names. 

In [ ]:
import csv
with open('feature_names.csv', 'w', newline='') as f: 
    wr = csv.writer(f)
    wr.writerow(X.columns.to_list())

In [ ]:
boto3.Session().resource('s3').Bucket(BUCKET).Object(os.path.join(prefix, 'data/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(BUCKET).Object(os.path.join(prefix, 'data/test.csv')).upload_file('test.csv')
boto3.Session().resource('s3').Bucket(BUCKET).Object(os.path.join(prefix, 'data/feature_names.csv')).upload_file('feature_names.csv')

## Training

In [ ]:
# Open Source distributed script mode
from sagemaker.session import s3_input, Session
from sagemaker.xgboost.estimator import XGBoost

# use validation set to choose # of trees
hyperparams = {
    "eta": 0.2,
    "max_depth": 4, 
    "subsample": 0.5, 
    "verbose": 0, 
    'num_round': 500
}

instance_type = "ml.c5.2xlarge"
output_path = 's3://{}/{}'.format(BUCKET, PREFIX)
content_type = "csv"
train_input = s3_input("s3://{}/{}/{}".format(BUCKET, PREFIX, 'data/train.csv'), content_type=content_type)
test_input = s3_input("s3://{}/{}/{}".format(BUCKET, PREFIX, 'data/test.csv'), content_type=content_type)
feature_names_input = s3_input("s3://{}/{}/{}".format(BUCKET, PREFIX, 'data/feature_names.csv'), content_type=content_type)

In [ ]:
boto_session = boto3.Session(region_name=region)
session = Session(boto_session=boto_session)
script_path = 'bikeshare-script.py'

xgb_script_mode_estimator = XGBoost(
    entry_point=script_path,
    framework_version='0.90-1', # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    train_instance_count=1, 
    train_instance_type=instance_type,
    output_path=output_path)

xgb_script_mode_estimator.fit({'train': train_input, 
                               'feature_names': feature_names_input,
                               'test': test_input})

Below script displays the bucket and path that contains the model artifact. 

In [ ]:
from urllib.parse import urlparse
MODEL_FILE_PATH = urlparse(xgb_script_mode_estimator.model_data, allow_fragments=False).path.lstrip('/')
print("Model artifact details \n BUCKET = '{}' \n MODEL_FILE_PATH = '{}'".format(BUCKET, MODEL_FILE_PATH))

# Interpretability for XGBoost models

### Setup

In [ ]:
!pip install -q xgboost graphviz shap

In [ ]:
import numpy as np
import boto3
import pickle as pkl    

s3 = boto3.resource('s3')
s3client = boto3.client('s3')
s3client.download_file(BUCKET, MODEL_FILE_PATH, 'model.tar.gz')
!tar -xvf model.tar.gz

Following parameters are used in the notebook visualization

In [ ]:
## model (XGBoost booster) file ## 
model = pkl.load(open('xgboost_model', 'rb'))
feature_names = model.feature_names
print(feature_names)

In [ ]:
## shap values ##
shap_values = pkl.load(open('shap_values', 'rb'))
N_ROWS = shap_values.shape[0]
N_FEATURES = shap_values.shape[1]

baseline_value = shap_values[0, -1]  # last column is the baseline
shap_values = np.delete(shap_values, -1, axis=1) # remove the last column

In [ ]:
## reference dataset used for SHAP computation ## 

# This dataset is assumed to *NOT* contain the output label 
# and have the same number of features as the shap_values
data = pkl.load(open('reference_data', 'rb'))

## Tree plot

An XGBoost model consists of an ensemble of classification and regression trees (CART).  Change the `NUM_TREE` value (ordinal number of the target tree) to plot other trees. 

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt 

NUM_TREE = 4

fig, ax = plt.subplots(figsize=(30, 30))
xgb.plot_tree(model, num_trees=NUM_TREE, ax=ax)
plt.show()

## XGBoost Feature importances

XGBoost provides multiple feature importance metrics to understand the influence of each feature on the model.

- *gain*: contribution of feature to the model calculated by taking the increase in prediction accuracy ('total_gain' is the sum of all gain across all splits, 'gain' is the average of all gain across all splits)

- *cover*: coverage of a feature calculated by the number of data points affected by a split involving the feature ('total_cover' is the sum of all coverage, 'cover' is the average across all splits)

- *weight*: percentage representing the relative number of times a particular feature occurs in the trees of the model.

In [ ]:
MAX_FEATURES = 10

fig = plt.figure(figsize=(18, 15))
fig.suptitle('Feature importances for the top {} features'.format(MAX_FEATURES), fontsize=24)

for index, type in enumerate(('gain', 'total_gain', 'cover', 'total_cover', 'weight'), start=1): 
    ax = fig.add_subplot(3, 2, index)
    xgb.plot_importance(model, importance_type=type, title=type,
                        ax=ax, grid=False, height=0.4, 
                        max_num_features=MAX_FEATURES, show_values=False)

## Explaining the model using SHAP

We use SHAP values as means to understand the contributions of the features to the model predictions. SHAP (SHapley Additive exPlanations) by Lundberg and Lee (2016) is a method to
explain individual predictions and is based on the game theoretically optimal
Shapley Values.

A prediction can be explained by assuming that each feature value of the
instance is a “player” in a game where the prediction is the payout. Shapley
values – a method from coalitional game theory – tells us how to fairly
distribute the “payout” among the features.

Reference: "Explainable machine-learning predictions for the prevention of hypoxaemia during surgery", Nature Biomedical Engineering, 2018

------

**Be careful to interpret the Shapley value correctly**: The Shapley value is the
average contribution of a feature value to the prediction in different
coalitions. The Shapley value is NOT the difference in prediction when we would
remove the feature from the model.

In [ ]:
import shap
shap.initjs()

### SHAP summary

A global aggregation of the individual Shapley values gives the overall average contributions of the features.

In [ ]:
shap.summary_plot(shap_values, feature_names=feature_names, plot_type="bar")

The **summary plot** below can provide more context over the bar chart of feature importances. It tells which features are most important, and also their range of effects over the dataset. The color allows us to match how changes in the value of a feature effect the change in risk.

In [ ]:
shap.summary_plot(shap_values, features=data, feature_names=feature_names)

### SHAP dependence plots

A **SHAP dependence plot** shows how the model output varies by feature value while showing the interaction between two features. The feature used for coloring is automatically chosen to highlight what might be driving these interactions. This shows how the model depends on the given feature, and can be considered a richer extenstion of the classical parital dependence plots. Vertical dispersion of the data points represents interaction effects. Grey ticks along the y-axis are data points where the feature’s value was NaN.

Here we plot the top ranked features  (ordered by mean absolute SHAP value over all the samples). 

In [ ]:
N_RANKS = 3
for r in range(N_RANKS):
    shap.dependence_plot("rank({})".format(r), 
                         shap_values, 
                         data, 
                         feature_names=feature_names,
                         interaction_index='auto')

### SHAP force plots

A **force plot** explanation shows how features are contributing to push the model output from the base value (the average model output over the dataset) to the actual prediction. Features pushing the prediction higher are shown in **red**, those pushing the prediction lower are in **blue**. 

In [ ]:
import numpy as np
N_ROWS = shap_values.shape[1]
N_SAMPLES = min(1000, N_ROWS)
sampled_indices = np.random.randint(N_ROWS, size=N_SAMPLES)

In [ ]:
shap.force_plot(baseline_value, 
                shap_values[sampled_indices, :], 
                data[sampled_indices, :], 
                feature_names=feature_names)

### Outliers

Outliers are extreme values that deviate from other observations on data. It's useful to understand the influence of various features for outlier predictions to determine if it's a novelty, an experimental error, or a shortcoming in the model. 

Here we show force plot for prediction outliers that are on either side of the baseline value. 

In [ ]:
# top outliers
from scipy import stats
N_OUTLIERS = 3  # number of outliers on each side of the tail

shap_sum = np.sum(shap_values, axis=1)
z_scores = stats.zscore(shap_sum)
outlier_indices = (np.argpartition(z_scores, -N_OUTLIERS)[-N_OUTLIERS:]).tolist()
outlier_indices += (np.argpartition(z_scores, N_OUTLIERS)[:N_OUTLIERS]).tolist()

In [ ]:
for fig_index, outlier_index in enumerate(outlier_indices, start=1): 
    shap.force_plot(baseline_value, 
                    shap_values[outlier_index, :], 
                    data[outlier_index, :], 
                    matplotlib=True, 
                    feature_names=feature_names) 